<a href="https://colab.research.google.com/github/Kevangi/summer-to-winter-cyclegan/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys

In [3]:
sys.path.append('/content/drive/MyDrive/Summer2Winter')

In [4]:
from dataset_loader import get_dataloader

In [5]:
dataset_path = "/content/drive/MyDrive/Summer2Winter/dataset"

In [6]:
dataloader = get_dataloader(dataset_path, batch_size=1)

In [7]:
batch = next(iter(dataloader))

In [8]:
print(batch["A"].shape)
print(batch["B"].shape)

torch.Size([1, 3, 256, 256])
torch.Size([1, 3, 256, 256])


In [9]:
from models.generator import Generator
from models.discriminator import Discriminator

In [10]:
import torch

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
G = Generator().to(device)
D = Discriminator().to(device)

In [13]:
real_image = batch["A"].to(device)

In [14]:
fake_image = G(real_image)
disc_output = D(fake_image)

In [15]:
print("Generator output shape:", fake_image.shape)
print("Discriminator output shape:", disc_output.shape)

Generator output shape: torch.Size([1, 3, 256, 256])
Discriminator output shape: torch.Size([1, 1, 30, 30])


In [16]:
import torch.nn as nn
import torch.optim as optim

In [17]:
D_A = Discriminator().to(device)  # Summer discriminator
D_B = Discriminator().to(device)  # Winter discriminator

In [18]:
# Adversarial loss (LSGAN)
criterion_GAN = nn.MSELoss()

In [19]:
# Cycle consistency loss
criterion_cycle = nn.L1Loss()

In [20]:
# Identity loss
criterion_identity = nn.L1Loss()

In [21]:
lambda_cycle = 10
lambda_identity = 5

In [23]:
optimizer_G = optim.Adam(
    list(G.parameters()) + list(D.parameters()),
    lr=0.0002,
    betas=(0.5, 0.999)
)

In [24]:
optimizer_D_A = optim.Adam(
    D_A.parameters(),
    lr=0.0002,
    betas=(0.5, 0.999)
)

In [25]:
optimizer_D_B = optim.Adam(
    D_B.parameters(),
    lr=0.0002,
    betas=(0.5, 0.999)
)

In [26]:
def get_valid_fake_labels(prediction):
    valid = torch.ones_like(prediction, device=device)
    fake = torch.zeros_like(prediction, device=device)
    return valid, fake